<h1>オライリー本の一章練習中</h1>
ドロップアウトによる、精度向上を試みる
ドロップアウトでは、全結合層を伝播する値を確率的に伝播させない

In [1]:
from __future__ import print_function
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.optimizers import SGD
from keras.utils import np_utils
from make_tensorboard import make_tensorboard

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


<h2>・訓練データとテストデータ読み込み（検証データの割合も設定）</h2>

In [2]:
np.random.seed(1671) #for reproducibility
#network and training
NB_EPOCH=20
BATCH_SIZE=128
VERBOSE=1
NB_CLASSES=10 #正解ラベルの数
OPTIMIZER=SGD() #SGD（確率的勾配降下法を用いる）
N_HIDDEN=128 #隠れ層の数
VALIDATION_SPLIT=0.2 #交差検定用の検証データを訓練用データから区切る割合
DROPOUT=0.3 #ドロップアウトさせる確率の設定

#mnistのデータセットを読み込む際に訓練データとテストデータがシャッフルされて振り分けられる
#y_trainは訓練用データ（X_train）のラベルを、y_testはテスト用データ（x_test）のラベルを意味する
(X_train,y_train),(X_test,y_test)=mnist.load_data()
print(X_train.shape[0],'訓練用データサンプル数')
print(X_test.shape[0],'テスト用サンプルデータ数')

#正解ラベルには「５」などが入っているが、それを0か1のバイナリデータに変換したものをラベルとしなければいけないkerasの仕組み
Y_train=np_utils.to_categorical(y_train,NB_CLASSES)
Y_test=np_utils.to_categorical(y_test,NB_CLASSES)

#X_train（訓練データ）は60000件で、1件分のデータ形式が28*28なので、データを60000件*784の一件あたりの次元数を一次元に整形し直す
RESHAPED=784
X_train=X_train.reshape(60000,RESHAPED)#784要素の一次元配列に変換
X_train=X_train.astype('float32')
X_test=X_test.reshape(10000,RESHAPED)
X_test=X_test.astype('float32')

#各要素が0〜1の値の範囲を取るように最大値255で除算し正規化を行う
X_train/=255
X_test/=255


60000 訓練用データサンプル数
10000 テスト用サンプルデータ数


<h2>モデルの作成</h2>

In [3]:
#10カテゴリに分類するため、出力層は10に設定
#出力層の最後にアクティベーション層のフトマックスで正規化
model=Sequential()
model.add(Dense(N_HIDDEN,input_shape=(RESHAPED,)))#第一引数に出力する数、第二引数（input_shape）に一次元化した配列の要素数である784を設定
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(N_HIDDEN))
model.add(Activation('relu'))
model.add(Dropout(DROPOUT))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               100480    
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
__________

<h2>モデルのコンパイル</h2>
<h4>モデルのコンパイルの際にいくつかオプションを与えることができる</h4>
<li>モデルの学習時の重み更新に使用する最適化アルゴリズムを選択することができる<br>
kerasで使える損失関数にはいくつか種類がある
MSE・・・平均二乗誤差（値が0〜１の範囲でなく、正解ラベルとの乖離が大きいと、大きく出力される）
バイナリクロスエントロピー・・・2値化分類の際に用いる
カテゴリカルクロスエントロピー・・・複数のクラスに対する損失関数を求めることができる。出力はデフォルトでsoftmax関数
<li>損失関数を選ぶ必要がある。最適化アルゴリズムが勾配を元に重み空間を最適な方向に導くために使用される<br>
<li>学習したモデルを評価する<br>
評価には精度、適合率、再現率、（それらを元に調和率）を求められる
精度・・・ターゲットに対する正誤
適合率・・・選択した項目がどれくらい複数クラス分類に関連しているかを示す
再現率・・・複数クラス分類において、特定のクラスに関する精度のいいを示す

In [4]:
model.compile(loss='categorical_crossentropy',
             optimizer=OPTIMIZER,#最初の方でSGDを設定済み
              metrics=['accuracy'])#精度で評価する

In [12]:
#学習過程をファイルとして出力するために別ファイルでTensorBoardを使う関数を定義済
callbacks=[make_tensorboard(set_dir_name='MNIST_test')]

<h2>モデルの学習を行う</h2>
<li>エポック回数の設定<br>
学習ごとに損失関数の値を選択された最適化アルゴリズムが最小値にするように重みを調整する
<li>バッチサイズ
最適化アルゴリズムが重みを更新する際に、データをいくつ使用するか設定

In [13]:
#バッチサイズ、エポック回数は最初の方に設定済
model.fit(X_train,Y_train,batch_size=BATCH_SIZE,epochs=NB_EPOCH,callbacks=callbacks,verbose=VERBOSE,validation_split=VALIDATION_SPLIT)

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 1s 28us/step - loss: 1.7139 - acc: 0.4568 - val_loss: 0.9383 - val_acc: 0.8022
Epoch 2/20
48000/48000 [==============================] - 1s 24us/step - loss: 0.9491 - acc: 0.7077 - val_loss: 0.5450 - val_acc: 0.8655
Epoch 3/20
48000/48000 [==============================] - 1s 24us/step - loss: 0.7095 - acc: 0.7825 - val_loss: 0.4289 - val_acc: 0.8877
Epoch 4/20
48000/48000 [==============================] - 1s 24us/step - loss: 0.6050 - acc: 0.8169 - val_loss: 0.3756 - val_acc: 0.8982
Epoch 5/20
48000/48000 [==============================] - 1s 24us/step - loss: 0.5392 - acc: 0.8394 - val_loss: 0.3409 - val_acc: 0.9038
Epoch 6/20
48000/48000 [==============================] - 1s 24us/step - loss: 0.4910 - acc: 0.8538 - val_loss: 0.3158 - val_acc: 0.9100
Epoch 7/20
48000/48000 [==============================] - 1s 25us/step - loss: 0.4616 - acc: 0.8621 - val_loss: 0.2985 - val_acc

<h2>モデルのテストを行う</h2>

In [14]:
score=model.evaluate(X_test,Y_test,verbose=VERBOSE)

10000/10000 [==============================] - 0s 23us/step


In [15]:
print("test score:",score[0])
print("test accuracy:",score[1])

test score: 0.19377726282924412
test accuracy: 0.9426


ドロップアウトによって、各ニューロンは近傍のニューロンを頼れなくなるため、各ニューロンが賢くなる
評価の際にはドロップアウトをもちいらないのは、学習で優れたニューロンを構築できたため